##推薦作法:
- 將需要的資料都存成變數：
    - 新聞內容：class(python裡class是保留字，所以我在下面的code改成category),title,newsDate,journalist,content,hit,url,source
    - 評論：url,commenter,comment
- 將這些變數用list或dictionary存起來
- 將存的list或dictionary轉成pandas
- 直接用DataFrame.to_csv(filename,encoding="utf-8")轉成csv
- 下面的code有存進資料庫的部分，如果不存資料庫的人可以把那些註解掉，存成csv就好，如果要存進資料庫，則記得先見好資料庫，資料庫的code詳見news.sql

## 抓取單一日期或日期區間  ETToday_day(startday,endday,sleep_time=0) 

In [13]:
# startday, endday 要輸入string(ex. '2015-1-1')
# 沒輸入日期預設抓取昨天新聞
# sleep_time意思為抓取每一天新聞之間要休息多久，單位為秒

import requests
from bs4 import BeautifulSoup
import math
import re
from datetime import date,datetime, timedelta 
import time

#################連資料庫用
def connect(dbname,password,host,user,port=3306):
    import MySQLdb
    # 承德電腦 host="10.120.30.1"
    # localhost host = '127.0.0.1'
    db = MySQLdb.connect(host=host,user="root",passwd=password,db=dbname,charset="utf8")
    return db
def insert_news(db,category=None,title=None,newsDate=None,journalist=None,content=None,hit=None,url=None,source=None):
    cur = db.cursor()
    cur.execute("insert into news(class,title,newsDate,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,journalist,content,hit,url,source])
    db.commit()
def insert_comments(db,url,commenter,comment,remark=None):
    cur = db.cursor()
    cur.execute("insert into comments(url,commenter,comment)values(%s,%s,%s)" ,[url,commenter,comment])
    db.commit()
def close(db):
    db.close()
#################連資料庫用
    
sentinel = object()
def ETToday_day(startday=(str(date.today()-timedelta(1))),endday=sentinel,sleep_time=0):
     
    if endday is sentinel:
        endday=startday
    
    def convert_day(d):
        return datetime.strptime(d, '%Y-%m-%d')
    
    ed=convert_day(endday)
    sd=convert_day(startday)

    assert ed >= sd, "endday must after startday"

    ds = []
    for i in xrange(0,(ed-sd).days+1):
        ds.append(str((ed- timedelta(i)).strftime('%Y-%m-%d')))
    
    for day in ds:
        urls_dic = {"政治":"http://www.ettoday.net/news/news-list-%s-1.htm"%day,
                    "財經":"http://www.ettoday.net/news/news-list-%s-17.htm"%day,
                    "國際":"http://www.ettoday.net/news/news-list-%s-2.htm"%day,
                    "大陸":"http://www.ettoday.net/news/news-list-%s-3.htm"%day,
                    "社會":"http://www.ettoday.net/news/news-list-%s-6.htm"%day,
                    "地方":"http://www.ettoday.net/news/news-list-%s-7.htm"%day,
                    "影劇":"http://www.ettoday.net/news/news-list-%s-9.htm"%day,
                    "3C":"http://www.ettoday.net/news/news-list-%s-20.htm"%day
                   }

        L = []
        for url in urls_dic:
            res = requests.get(urls_dic[url])
            res.encoding="utf-8"
            soup = BeautifulSoup(res.text)
            page = re.search('\((\d+).\)',soup.select(".info")[0].text).group(1)
            pageCount = int(math.ceil(float(page)/30))
            for block in soup.select("#all-news-list > h3"):
                title=block.select('a')[0].text
                aLink=block.select('a')[0]['href']
                content = soup.select('.story > sectione > p')[1:]
                for i in xrange(0,len(content)):
                    for con in content[i].select('img'):
                        content[i].img.decompose()
                jour = str(soup.select('.story > sectione > p')[0].text.encode('utf-8'))
                journalist = (jour.split("記者")[1]).split("／")[0]
                L.append({'Category':block.select('em')[0].text,'Title':title,'Content':content,'Journalist':journalist,'Link':aLink,'Time':day,'Source':'ETToday'})
                #################連資料庫用
                connect("news","","localhost","root")
                insert_news()
                close()
                connect("comments","","localhost","root")
                insert_comments()
                close()
                #################連資料庫用
                time.sleep(sleep_time)
            
            if pageCount > 1:
                for i in xrange(2,pageCount+1):
                    url_1=str(urls_dic[url]).split(".htm")[0] + "-" + str(i) + ".htm"
                    res = requests.get(url_1)
                    res.encoding="utf-8"
                    soup = BeautifulSoup(res.text)
                    for block in soup.select("#all-news-list > h3"):
                        title=block.select('a')[0].text
                        aLink=block.select('a')[0]['href']
                        content = soup.select('.story > sectione > p')[1:]
                        for i in xrange(0,len(content)):
                            for con in content[i].select('img'):
                                content[i].img.decompose()
                        jour = str(soup.select('.story > sectione > p')[0].text.encode('utf-8'))
                        journalist = (jour.split("記者")[1]).split("／")[0]
                        L.append({'Category':block.select('em')[0].text,'Title':title,'Content':content,'Journalist':journalist,'Link':aLink,'Time':day,'Source':'ETToday'})
                        #################連資料庫用
                        connect("news","","localhost","root")
                        insert_news()
                        close()
                        connect("comments","","localhost","root")
                        insert_comments()
                        close()
                        #################連資料庫用
                        time.sleep(sleep_time)
            
    # pandas
    import pandas as pd
    df = pd.DataFrame(L,columns =['Category','Title','Content','Journalist','Link','Time','Source'])

    # write in csv
    if startday==endday:
        std="".join(startday.split('-'))
        filename="ETToday"+"_"+std+".csv"
    else:
        std="".join(startday.split('-'))
        end="".join(endday.split('-'))
        filename="ETToday"+"_"+std+"-"+end+".csv"
    print filename
    f=open(filename,"w")
    df.to_csv(filename,encoding="utf-8")
    f.close()


In [18]:
# run
ETToday_day('2015-06-04')

TypeError: must be string, not object

## 抓取單一或多月份  ETToday_month(year,month,year2,month2,sleep_time=0) 

In [14]:
# sleep_time意思為抓取每一天新聞之間要休息多久，單位為秒
# year, month要輸入integer(ex. 2012,3)
# 如果抓本月的新聞，會預設抓到今天前一天的新聞

import requests
from bs4 import BeautifulSoup
import math
import re
from datetime import date,datetime, timedelta 
import time
import calendar

#################連資料庫用
def connect(dbname,password,host,user,port=3306):
    import MySQLdb
    # 承德電腦 host="10.120.30.1"
    # localhost host = '127.0.0.1'
    db = MySQLdb.connect(host=host,user="root",passwd=password,db=dbname,charset="utf8")
    return db
def insert_news(db,category=None,title=None,newsDate=None,journalist=None,content=None,hit=None,url=None,source=None):
    cur = db.cursor()
    cur.execute("insert into news(class,title,newsDate,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,journalist,content,hit,url,source])
    db.commit()
def insert_comments(db,url,commenter,comment,remark=None):
    cur = db.cursor()
    cur.execute("insert into comments(url,commenter,comment)values(%s,%s,%s)" ,[url,commenter,comment])
    db.commit()
def close(db):
    db.close()
#################連資料庫用

sentinel = object()
def ETToday_month(year,month,year2=sentinel,month2=sentinel,sleep_time=0):
    
    if year2 is sentinel:
        year2 = year
    if month2 is sentinel:
        month2 = month
    
    assert date(year2,month2,1)>=date(year,month,1), "year2,month2 must after year,month"

    def first_day_of_month_d(d):
        return date(d.year, d.month, 1)

    def first_day_of_month(y,m):
        return date(y,m,1)

    def last_day_of_month(y,m):
        return date(y,m+1,1) - timedelta(days = 1)

    def diff_months_d(d1, d2):
        return (d1.year - d2.year)*12 + d1.month - d2.month

    def diff_days(y,m,y2,m2):
        return abs(last_day_of_month(y2,m2)-first_day_of_month(y,m)).days

    # check if input month exceed current month
    assert diff_months_d(date.today(),first_day_of_month(year,month))>=0, "眼睛睜大點好嘛？%s月還沒到ok？"%month
    assert diff_months_d(date.today(),first_day_of_month(year2,month2))>=0, "眼睛睜大點好嘛？%s月還沒到ok？"%month2

    if diff_months_d(date.today(),first_day_of_month(year2,month2))==0:
        duration=(date.today()-first_day_of_month_d(date.today())).days
        ds = []
        for i in xrange(1,duration+1):
            ds.append(str((date.today()- timedelta(i)).strftime('%Y-%m-%d')))
    else:
        #duration=calendar.monthrange(year,month)[1]
        duration=diff_days(year,month,year2,month2)
        ds = []
        for i in xrange(0,duration+1):
            ds.append(str((last_day_of_month(year2,month2)- timedelta(i)).strftime('%Y-%m-%d')))

    for day in ds:
        urls_dic = {"政治":"http://www.ettoday.net/news/news-list-%s-1.htm"%day,
                    "財經":"http://www.ettoday.net/news/news-list-%s-17.htm"%day,
                    "國際":"http://www.ettoday.net/news/news-list-%s-2.htm"%day,
                    "大陸":"http://www.ettoday.net/news/news-list-%s-3.htm"%day,
                    "社會":"http://www.ettoday.net/news/news-list-%s-6.htm"%day,
                    "地方":"http://www.ettoday.net/news/news-list-%s-7.htm"%day,
                    "影劇":"http://www.ettoday.net/news/news-list-%s-9.htm"%day,
                    "3C":"http://www.ettoday.net/news/news-list-%s-20.htm"%day
                   }

        L = []
        for url in urls_dic:
            res = requests.get(urls_dic[url])
            res.encoding="utf-8"
            soup = BeautifulSoup(res.text)
            page = re.search('\((\d+).\)',soup.select(".info")[0].text).group(1)
            pageCount = int(math.ceil(float(page)/30))
            for block in soup.select("#all-news-list > h3"):
                title=block.select('a')[0].text
                aLink=block.select('a')[0]['href']
                content = soup.select('.story > sectione > p')[1:]
                for i in xrange(0,len(content)):
                    for con in content[i].select('img'):
                        content[i].img.decompose()
                jour = str(soup.select('.story > sectione > p')[0].text.encode('utf-8'))
                journalist = (jour.split("記者")[1]).split("／")[0]
                L.append({'Category':block.select('em')[0].text,'Title':title,'Content':content,'Journalist':journalist,'Link':aLink,'Time':day,'Source':'ETToday'})
                #################連資料庫用
                connect("news","","localhost","root")
                insert_news()
                close()
                connect("comments","","localhost","root")
                insert_comments()
                close()
                #################連資料庫用
                time.sleep(sleep_time)
            
            if pageCount > 1:
                for i in xrange(2,pageCount+1):
                    url_1=str(urls_dic[url]).split(".htm")[0] + "-" + str(i) + ".htm"
                    res = requests.get(url_1)
                    res.encoding="utf-8"
                    soup = BeautifulSoup(res.text)
                    for block in soup.select("#all-news-list > h3"):
                        title=block.select('a')[0].text
                        aLink=block.select('a')[0]['href']
                        content = soup.select('.story > sectione > p')[1:]
                        for i in xrange(0,len(content)):
                            for con in content[i].select('img'):
                                content[i].img.decompose()
                        jour = str(soup.select('.story > sectione > p')[0].text.encode('utf-8'))
                        journalist = (jour.split("記者")[1]).split("／")[0]
                        L.append({'Category':block.select('em')[0].text,'Title':title,'Content':content,'Journalist':journalist,'Link':aLink,'Time':day,'Source':'ETToday'})
                        #################連資料庫用
                        connect("news","","localhost","root")
                        insert_news()
                        close()
                        connect("comments","","localhost","root")
                        insert_comments()
                        close()
                        #################連資料庫用
                        time.sleep(sleep_time)
            
    # pandas
    import pandas as pd
    df = pd.DataFrame(L,columns =['Category','Title','Content','Journalist','Link','Time','Source'])

    # write in csv
    if (str(year)+str(month))==(str(year2)+str(month2)):
        filename="ETTdoay_%d%02d.csv"%(year,month)
    else:
        filename="ETTdoay_%d%02d-%d%02d.csv"%(year,month,year2,month2)
    f=open(filename,"w")
    df.to_csv(filename,encoding="utf-8")
    f.close()

In [ ]:
# run
ETToday_month(2015,5)

##直接輸入從今天算起要抓幾天 ETToday(days,sleep_time=0)

In [396]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import math
import re
from datetime import date,datetime, timedelta 
import time

#################連資料庫用
def connect(dbname,password,host,user,port=3306):
    import MySQLdb
    # 承德電腦 host="10.120.30.1"
    # localhost host = '127.0.0.1'
    db = MySQLdb.connect(host=host,user="root",passwd=password,db=dbname,charset="utf8")
    return db
def insert_news(db,category=None,title=None,newsDate=None,newsTime=None,journalist=None,content=None,hit=None,url=None,source="ETToday"):
    cur = db.cursor()
    cur.execute("insert into news(class,title,newsDate,newsTime,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,newsTime,journalist,content,hit,url,source])
    db.commit()
def insert_comments(db,url,commenter,comment,remark=None):
    cur = db.cursor()
    cur.execute("insert into comments(commentDate,commentTime,commenter,comment,url)values(%s,%s,%s,%s,%s)" ,[commentDate,commentTime,url,commenter,comment])
    db.commit()
def close(db):
    db.close()
#################連資料庫用
    
def ETToday(days,sleep_time=0,category=["政治","財經","國際","大陸"]):
    #day = str(date.today()- timedelta(1))
    
    hit = None
    ds = []
    for i in xrange(1,days+1):
        ds.append(str(date.today()- timedelta(i)))
    
    for day in ds:
        url_dic = {"政治":"http://www.ettoday.net/news/news-list-%s-1.htm"%day,
                    "財經":"http://www.ettoday.net/news/news-list-%s-17.htm"%day,
                    "國際":"http://www.ettoday.net/news/news-list-%s-2.htm"%day,
                    "大陸":"http://www.ettoday.net/news/news-list-%s-3.htm"%day,
                    "社會":"http://www.ettoday.net/news/news-list-%s-6.htm"%day,
                    "地方":"http://www.ettoday.net/news/news-list-%s-7.htm"%day,
                    "影劇":"http://www.ettoday.net/news/news-list-%s-9.htm"%day,
                    "3C":"http://www.ettoday.net/news/news-list-%s-20.htm"%day
                   }
        
        url_categories = {}
        for cat in category:
            url_categories[cat]=url_dic[cat]

        L = []
        for cat in url_categories:
            res = requests.get(url_categories[cat])
            res.encoding="utf-8"
            soup = BeautifulSoup(res.text)
            page = re.search('\((\d+).\)',soup.select(".info")[0].text).group(1)
            pageCount = int(math.ceil(float(page)/30))

            for block in soup.select("#all-news-list > h3"):
                title=block.select('a')[0].text
                aLink=block.select('a')[0]['href']
                driver = webdriver.PhantomJS()
                driver.get(aLink)
                
                #########get page breakpoint
                try:
                    WebDriverWait(driver, 10).until(
                            EC.visibility_of_element_located((By.CLASS_NAME, "story"))
                        )
                    html = driver.page_source
                    cont = str(html.encode('utf-8'))
                    soupCont = BeautifulSoup(cont)
                except Exception as e:
                    f=open("ETTodayException.log","a")
                    ExList=[
                        "Log at: " + str(datetime.today()) + "\n",
                        "\t -News Date: " + day + "\n",
                        "\t -News Category: " + cat + "\n",
                        "\t -News url: " + aLink + "\n",
                        "\t -BreakPoint: get page" + "\n",
                        "\t -Exception Type: " + str(e.__class__) + "\n",
                        "\t -Exception Message: " + e.message + "\n"
                    ]
                    f.writelines(ExList)
                    f.close
                finally:
                    driver.quit()
                #########get page breakpoint
                
                #########content breakpoint
                try:
                    content = soupCont.select('.story > sectione > p')[1:]
                except Exception as e:
                    f=open("ETTodayException.log","a")
                    ExList=[
                        "Log at: " + str(datetime.today()) + "\n",
                        "\t -News Date: " + day + "\n",
                        "\t -News Category: " + cat + "\n",
                        "\t -News url: " + aLink + "\n",
                        "\t -BreakPoint: content" + "\n",
                        "\t -Exception Type: " + str(e.__class__) + "\n",
                        "\t -Exception Message: " + e.message + "\n"
                    ]
                    f.writelines(ExList)
                    f.close
                #########content breakpoint
                
                try:
                    for i in xrange(0,len(content)):
                        for con in content[i].select('img'):
                            content[i].img.decompose()
                except:
                    pass

                #########journist breakpoint
                try:
                    try:
                        journalist=re.search('<p>([^圖].*?)／.*?報導<',cont).group(1)
                    except:
                        pass
                    try:
                        journalist=re.search('>文／(.*?)<',cont).group(1)
                    except:
                        pass
                except Exception as e:
                    f=open("ETTodayException.log","a")
                    ExList=[
                        "Log at: " + str(datetime.today()) + "\n",
                        "\t -News Date: " + day + "\n",
                        "\t -News Category: " + cat + "\n",
                        "\t -News url: " + aLink + "\n",
                        "\t -BreakPoint: journist" + "\n",
                        "\t -Exception Type: " + str(e.__class__) + "\n",
                        "\t -Exception Message: " + e.message + "\n"
                    ]
                    f.writelines(ExList)
                    f.close
                #########journist breakpoint
                
                #########category breakpoint
                category=block.select('em')[0].text
                if not category == cat:
                    try:
                        raise Exception
                    except Exception as e:
                        f=open("ETTodayException.log","a")
                        ExList=[
                            "Log at: " + str(datetime.today()) + "\n",
                            "\t -News Date: " + day + "\n",
                            "\t -News Category: " + cat + "\n",
                            "\t -News url: " + aLink + "\n",
                            "\t -BreakPoint: category" + "\n",
                            "\t -Exception Type: " + str(e.__class__) + "\n",
                            "\t -Exception Message: " + e.message + "\n"
                        ]
                        f.writelines(ExList)
                        f.close
                #########category breakpoint
                
                L.append({'Category':category,'Title':title,'Content':content,'Journalist':journalist,'Link':aLink,'Time':day,'Source':'ETToday'})
                
                #################連資料庫用
                conn=connect("practice","oblivn0374","localhost","root")
                insert_news(conn)
                #insert_comments(conn)
                close(conn)
                #################連資料庫用
                time.sleep(sleep_time)
            
            if pageCount > 1:
                for i in xrange(2,pageCount+1):
                    cat_1=str(url_categories[cat]).split(".htm")[0] + "-" + str(i) + ".htm"
                    res = requests.get(cat_1)
                    res.encoding="utf-8"
                    soup = BeautifulSoup(res.text)
                    for block in soup.select("#all-news-list > h3"):
                        title=block.select('a')[0].text
                        aLink=block.select('a')[0]['href']
                        driver = webdriver.PhantomJS()
                        driver.get(aLink)
                        
                        #########get page breakpoint
                        try:
                            WebDriverWait(driver, 10).until(
                                    EC.visibility_of_element_located((By.CLASS_NAME, "story"))
                                )
                            html = driver.page_source
                            cont = str(html.encode('utf-8'))
                            soupCont = BeautifulSoup(cont)
                        except Exception as e:
                            f=open("ETTodayException.log","a")
                            ExList=[
                                "Log at: " + str(datetime.today()) + "\n",
                                "\t -News Date: " + day + "\n",
                                "\t -News Category: " + cat + "\n",
                                "\t -News url: " + aLink + "\n",
                                "\t -BreakPoint: get page" + "\n",
                                "\t -Exception Type: " + str(e.__class__) + "\n",
                                "\t -Exception Message: " + e.message + "\n"
                            ]
                            f.writelines(ExList)
                            f.close
                        finally:
                            driver.quit()
                        #########get page breakpoint    
                            
                        #########content breakpoint
                        try:
                            content = soupCont.select('.story > sectione > p')[1:]
                        except Exception as e:
                            f=open("ETTodayException.log","a")
                            ExList=[
                                "Log at: " + str(datetime.today()) + "\n",
                                "\t -News Date: " + day + "\n",
                                "\t -News Category: " + cat + "\n",
                                "\t -News url: " + aLink + "\n",
                                "\t -BreakPoint: content" + "\n",
                                "\t -Exception Type: " + str(e.__class__) + "\n",
                                "\t -Exception Message: " + e.message + "\n"
                            ]
                            f.writelines(ExList)
                            f.close
                        #########content breakpoint
                        
                        try:
                            for i in xrange(0,len(content)):
                                for con in content[i].select('img'):
                                    content[i].img.decompose()
                        except:
                            pass
                        
                        #########journist breakpoint
                        try:
                            try:
                                journalist=re.search('<p>([^圖].*?)／.*?報導<',cont).group(1)
                            except:
                                pass
                            try:
                                journalist=re.search('>文／(.*?)<',cont).group(1)
                            except:
                                pass
                        except Exception as e:
                        f=open("ETTodayException.log","a")
                        ExList=[
                            "Log at: " + str(datetime.today()) + "\n",
                            "\t -News Date: " + day + "\n",
                            "\t -News Category: " + cat + "\n",
                            "\t -News url: " + aLink + "\n",
                            "\t -BreakPoint: journist" + "\n",
                            "\t -Exception Type: " + str(e.__class__) + "\n",
                            "\t -Exception Message: " + e.message + "\n"
                        ]
                        f.writelines(ExList)
                        f.close
                        #########journist breakpoint
                        
                        category=block.select('em')[0].text
                        L.append({'Category':category,'Title':title,'Content':content,'Journalist':journalist,'Link':aLink,'Time':day,'Source':'ETToday'})
                        
                        #################連資料庫用
                        conn=connect("practice","oblivn0374","localhost","root")
                        insert_news(conn,category,title,day,journalist,content,hit,alink)
                        #insert_comments(conn)
                        close(conn)
                        #################連資料庫用
                        time.sleep(sleep_time)
            
    # pandas
    import pandas as pd
    df = pd.DataFrame(L,columns =['Category','Title','Content','Journalist','Link','Time','Source'])

    # write in csv
    f=open("ETToday.csv","w")
    df.to_csv("ETToday",encoding="utf-8")
    f.close()


In [397]:
# run
ETToday(1)

TimeoutException: Message: 
Screenshot: available via screen


In [ ]:
def connect(dbname,password,host,user,port=3306):
    import MySQLdb
    # 承德電腦 host="10.120.30.1"
    # localhost host = '127.0.0.1'
    db = MySQLdb.connect(host=host,user="root",passwd=password,db=dbname,charset="utf8")
    return db
def insert_news(db,category=None,title=title,newsDate=day,journalist=journalist,content=content,hit=hit,url=aLink,source="ETToday"):
    cur = db.cursor()
    cur.execute("insert into news(class,title,newsDate,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,journalist,content,hit,url,source])
    db.commit()
def insert_comments(db,url,commenter,comment,remark=None):
    cur = db.cursor()
    cur.execute("insert into comments(url,commenter,comment)values(%s,%s,%s)" ,[url,commenter,comment])
    db.commit()
def close(db):
    db.close()

In [269]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import math
import re
from datetime import date,datetime, timedelta 
import time
import pandas as pd

#################連資料庫用
def connect(dbname,password,host,user,port=3306):
    import MySQLdb
    # 承德電腦 host="10.120.30.1"
    # localhost host = '127.0.0.1'
    db = MySQLdb.connect(host=host,user="root",passwd=password,db=dbname,charset="utf8")
    return db
def insert_news(db,category=None,title=None,newsDate=None,newsTime=None,journalist=None,content=None,hit=None,url=None,source="ETToday"):
    cur = db.cursor()
    cur.execute("insert into news(class,title,newsDate,newsTime,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,newsTime,journalist,content,hit,url,source])
    db.commit()
def insert_comments(db,commentDate,commentTime,commenter,commenterUrl,comment,url,remark=None):
    cur = db.cursor()
    cur.execute("insert into comments(commentDate,commentTime,commenter,commenterUrl,comment,url)values(%s,%s,%s,%s,%s,%s)" ,[commentDate,commentTime,commenter,commenterUrl,comment,url])
    db.commit()
def insert_newsRaw(db,url,newsRaw):
    cur = db.cursor()
    cur.execute("insert into newsRaw(url,newsRaw)values(%s,%s)",[url,newsRaw])
    db.commit()
def close(db):
    db.close()
#################連資料庫用
    
def ETToday(days=1,sleep_time=0,category=["政治","財經","國際","大陸"]):
    
    ds = []
    for i in xrange(1,days+1):
        ds.append(str(date.today()- timedelta(i)))
    
    for newsDate in ds:
        url_dic = {"政治":"http://www.ettoday.net/news/news-list-%s-1.htm"%newsDate,
                    "財經":"http://www.ettoday.net/news/news-list-%s-17.htm"%newsDate,
                    "國際":"http://www.ettoday.net/news/news-list-%s-2.htm"%newsDate,
                    "大陸":"http://www.ettoday.net/news/news-list-%s-3.htm"%newsDate,
                    "社會":"http://www.ettoday.net/news/news-list-%s-6.htm"%newsDate,
                    "地方":"http://www.ettoday.net/news/news-list-%s-7.htm"%newsDate,
                    "影劇":"http://www.ettoday.net/news/news-list-%s-9.htm"%newsDate,
                    "3C":"http://www.ettoday.net/news/news-list-%s-20.htm"%newsDate
                   }
        
        url_categories = {}
        for cat in category:
            url_categories[cat]=url_dic[cat]

        L = []
        for cat in url_categories:
            try:
                res = requests.get(url_categories[cat])
                res.encoding="utf-8"
                soup = BeautifulSoup(res.text)
                page = re.search('\((\d+).\)',soup.select(".info")[0].text).group(1)
                pageCount = int(math.ceil(float(page)/30))
                soups = [soup]
            except:
                print url_categories[cat]
                raise 
            if pageCount >1:
                for i in xrange(2,pageCount+1):
                    curl = str(url_categories[cat]).split(".htm")[0] + "-" + str(i) + ".htm"
                    res = requests.get(curl)
                    res.encoding="utf-8"
                    soups.append(BeautifulSoup(res.text))

            for soup in soups:         
                for block in soup.select("#all-news-list > h3"):
                    try:
                        timeRegex=re.search("\[(\d\d)\/(\d\d) (\d{1,2}\:\d{1,2})\]",block.select('span')[0].text)
                        testDate=timeRegex.group(1)+'-'+timeRegex.group(2)
                        if not testDate == re.search('\d{4}-(\d\d-\d\d)',newsDate).group(1).decode('utf-8'):
                            continue
                        newsTime=timeRegex.group(3)
                        title=block.select('a')[0].text
                        aLink=block.select('a')[0]['href']
                    except:
                        print url_categories[cat]
                        raise
                    #########get page breakpoint
                    #driver = webdriver.Firefox()
                    driver = webdriver.Chrome()
                    #driver = webdriver.PhantomJS()
                    driver.get(aLink)
                    try:
                        WebDriverWait(driver, 10).until(
                                EC.visibility_of_element_located((By.ID, "fbComments"))
                            )
                        html = driver.page_source
                        cont = str(html.encode('utf-8'))
                        soupCont = BeautifulSoup(cont)
                    except Exception as e:
                        f=open("ETTodayException.log","a")
                        ExList=[
                            "Log at: " + str(datetime.today()) + "\n",
                            "\t -News Date: " + newsDate + "\n",
                            "\t -News Category: " + cat + "\n",
                            "\t -News url: " + aLink + "\n",
                            "\t -BreakPoint: get page" + "\n",
                            "\t -Exception Type: " + str(e.__class__) + "\n",
                            "\t -Exception Message: " + e.message + "\n"
                        ]
                        f.writelines(ExList)
                        f.close
                    finally:
                        driver.quit()
                    #########get page breakpoint

                    #########content breakpoint
                    try:
                        content=""
                        contentRaw = soupCont.select('.story > sectione > p')
                        for p in contentRaw:
                            content = content + p.text
                    except Exception as e:
                        f=open("ETTodayException.log","a")
                        ExList=[
                            "Log at: " + str(datetime.today()) + "\n",
                            "\t -News Date: " + newsDate + "\n",
                            "\t -News Category: " + cat + "\n",
                            "\t -News url: " + aLink + "\n",
                            "\t -BreakPoint: content" + "\n",
                            "\t -Exception Type: " + str(e.__class__) + "\n",
                            "\t -Exception Message: " + e.message + "\n"
                        ]
                        f.writelines(ExList)
                        f.close
                    #########content breakpoint

                    try:
                        for i in xrange(0,len(content)):
                            for con in content[i].select('img'):
                                content[i].img.decompose()
                    except:
                        pass

                    #########journist breakpoint
                    try:
                        try:
                            journalist=re.search('<p>([^圖].*?)／.*?報導<',cont).group(1)
                        except:
                            pass
                        try:
                            journalist=re.search('>文／(.*?)<',cont).group(1)
                        except:
                            pass
                    except Exception as e:
                        f=open("ETTodayException.log","a")
                        ExList=[
                            "Log at: " + str(datetime.today()) + "\n",
                            "\t -News Date: " + newsDate + "\n",
                            "\t -News Category: " + cat + "\n",
                            "\t -News url: " + aLink + "\n",
                            "\t -BreakPoint: journist" + "\n",
                            "\t -Exception Type: " + str(e.__class__) + "\n",
                            "\t -Exception Message: " + e.message + "\n"
                        ]
                        f.writelines(ExList)
                        f.close
                    #########journist breakpoint
                    
                    #########comment breakpoint
                    comments=[]
                    try:
                        commLink = soupCont.select('.fb-comments.fb_iframe_widget > span > iframe')[0]['src']
                        resComm = requests.get(commLink)
                        resComm.encoding = "utf-8"
                        soupComm = BeautifulSoup(resComm.text)      
                        commentBlock=soupComm.select('.postContainer.fsl.fwb.fcb')
                        for comm in commentBlock: 
                            dt=re.search("(\d\d\d\d).(\d{1,2}).(\d{1,2}). (\d{1,2}\:\d{1,2})",comm.select('abbr')[0]['title'])
                            commentDate="%s-%02d-%02d"%(dt.group(1),int(dt.group(2)),int(dt.group(3)))
                            commentTime=dt.group(4)
                            commenter=comm.select('.profileName')[0].text
                            commenterUrl=comm.select('.profileName')[0]['href']
                            comment=comm.select('.postText')[0].text
                            comments.append({'commentDate':commentDate,'commentTime':commentTime,'commenter':commenter,'commenterUrl':commenterUrl,'comment':comment,'url':aLink,'remark':None})
                    except Exception as e:
                        f=open("ETTodayException.log","a")
                        ExList=[
                            "Log at: " + str(datetime.today()) + "\n",
                            "\t -News Date: " + newsDate + "\n",
                            "\t -News Category: " + cat + "\n",
                            "\t -News url: " + aLink + "\n",
                            "\t -BreakPoint: comment" + "\n",
                            "\t -Exception Type: " + str(e.__class__) + "\n",
                            "\t -Exception Message: " + e.message + "\n"
                        ]
                        f.writelines(ExList)
                        f.close
                    #########comment breakpoint

                    #########category breakpoint
                    category=block.select('em')[0].text
                    if not category == cat.decode("utf-8"):
                        try:
                            raise Exception
                        except Exception as e:
                            f=open("ETTodayException.log","a")
                            ExList=[
                                "Log at: " + str(datetime.today()) + "\n",
                                "\t -News Date: " + newsDate + "\n",
                                "\t -News Category: " + cat + "\n",
                                "\t -News url: " + aLink + "\n",
                                "\t -BreakPoint: category" + "\n",
                                "\t -Exception Type: " + str(e.__class__) + "\n",
                                "\t -Exception Message: " + e.message + "\n"
                            ]
                            f.writelines(ExList)
                            f.close
                    #########category breakpoint
    
                    try:
                        L.append({'class':category,'title':title,'newsDate':newsDate,'newsTime':newsTime,'journalist':journalist,'content':content,'hit':hit,'url':aLink,'source':'ETToday'})
                    except:
                        pass
                    
                    #################連資料庫用
                    hit = None
                    #print category,title,newsDate,newsTime,journalist,content,hit,aLink,commentDate,commentTime,commenter,comment
                    try:
                        conn=connect("practice","oblivn0374","localhost","root")
                        insert_news(conn,category,title,newsDate,newsTime,journalist,content[0],hit,aLink)
                        insert_comments(conn,commentDate,commentTime,commenter,commenterUrl,comment,aLink)
                        insert_newsRaw(conn,aLink,res.text)
                        close(conn)
                    except:
                        print "nothing write in DB"
                    #################連資料庫用
                    time.sleep(sleep_time)
    
    
    #########write_in_csv breakpoint
    try:
        # pandas - content
        df = pd.DataFrame(L,columns =['class','title','newsDate','newsTime','journalist','content','hit','url','source'])
        # pandas - comment
        df_comm = pd.DataFrame(comments,columns =['commentDate','commentTime','commenter','commenterUrl','comment','url'])

        # write content in csv
        f=open("ETToday_content.csv","w")
        df.to_csv("ETToday_content.csv",encoding="utf-8")
        f.close()
        # write comment in csv
        f=open("ETToday_comment.csv","w")
        df_comm.to_csv("ETToday_comment.csv",encoding="utf-8")
        f.close()
    except Exception as e:
        f=open("ETTodayException.log","a")
        ExList=[
            "Log at: " + str(datetime.today()) + "\n",
            "\t -News Date: " + newsDate + "\n",
            "\t -News Category: " + cat + "\n",
            "\t -News url: " + aLink + "\n",
            "\t -BreakPoint: write_in_csv" + "\n",
            "\t -Exception Type: " + str(e.__class__) + "\n",
            "\t -Exception Message: " + e.message + "\n"
        ]
        f.writelines(ExList)
        f.close
#########write_in_csv breakpoint

    

In [212]:
ETToday(category=["政治"])

nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB
nothing write in DB


AttributeError: 'str' object has no attribute 'select'

## For Test

In [263]:
type(re.search('\d{4}-(\d\d-\d\d)',newsDate).group(1).decode('utf-8'))

unicode

In [262]:
type(testDate[0])

unicode

In [ ]:
import requests
url="https://www.facebook.com/yijuamy.chen"
headers={
':host':'www.facebook.com',
':method':'GET',
':path':'/yijuamy.chen',
':scheme':'https',
':version':'HTTP/1.1',
'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
'accept-encoding':'gzip, deflate, sdch',
'accept-language':'en-US,en;q=0.8,zh-CN;q=0.6,zh;q=0.4',
'cache-control':'max-age=0',
'cookie':'datr=QBNeUwqI4S51V5AzGEoeCA1r; _ga=GA1.2.1275457108.1428803544; lu=TgMfpfq2tjUWZNCGMvAtPqgg; c_user=100000310235880; fr=0R1TtmugtGGZqdare.AWXzmmSln6UThlEI7X7DWth-8LI.BT6iv-.MD.FV-.0.AWXKXYOq; xs=46%3AcvZCripsh73rfw%3A2%3A1434196224%3A14635; csm=2; s=Aa5VbbutJxLdjkjE.BVfBkA; act=1434791149057%2F13; p=-2; presence=EM434791329EuserFA21B00310235880A2EstateFDsb2F0Et2F_5b_5dElm2FnullEuct2F1434788575BEtrFA2loadA2EtwF2521827791EatF1434791328792G434791159662CEchFDp_5f1B00310235880F0CC; wd=1239x259',
'referer':'https://www.facebook.com/plugins/comments.php?api_key=146858218737386&channel_url=http%3A%2F%2Fstatic.ak.facebook.com%2Fconnect%2Fxd_arbiter%2F1ldYU13brY_.js%3Fversion%3D41%23cb%3Df3be262b0c%26domain%3Dwww.ettoday.net%26origin%3Dhttp%253A%252F%252Fwww.ettoday.net%252Ffc8173f04%26relation%3Dparent.parent&colorscheme=light&href=http%3A%2F%2Fwww.ettoday.net%2Fnews%2F20150612%2F520021.htm&locale=zh_TW&numposts=10&sdk=joey&skin=light&version=v2.0&width=620',
'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.124 Safari/537.36'
}

